### First bring in the necessary import statements to EDA and initial modeling to examine the data.

In [2]:
import re
import nltk
import spacy
import string
import imblearn
import numpy as np
import pandas as pd
import seaborn as sns
import texthero as hr
from PIL import Image 
from wordcloud import WordCloud
from nltk import pos_tag, FreqDist
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from sklearn.manifold import TSNE
from collections import defaultdict
from mpl_toolkits.mplot3d import Axes3D
from nltk.tokenize import TweetTokenizer
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from matplotlib.ticker import MaxNLocator
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction import text
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from imblearn.pipeline import Pipeline as imbPipeline
from nltk.stem import WordNetLemmatizer, PorterStemmer
from imblearn.metrics import classification_report_imbalanced
from nltk.tokenize import word_tokenize, regexp_tokenize, RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, AdaBoostClassifier
from sklearn.metrics import plot_roc_curve, plot_confusion_matrix, confusion_matrix, classification_report, accuracy_score, precision_score

import os
import sys
sys.setrecursionlimit(100000)
module_path = os.path.abspath(os.pardir)
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
%matplotlib inline
pd.set_option('display.max_colwidth', None)

/Users/jax/Documents/Flatiron


### First model I'd like to use a count vectorizer and good ole logistic regression. Then I'll use naive bayes which is particularly successful with nlp because it assumes *naively* there is no interdependence amongst the variables. Also Multinomial Bayes allows me to train my model with less data and potentionally mislabeled data. So first let's split our data.

In [4]:
unique_df = pd.read_csv('data/modeling_tweets.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/modeling_tweets.csv'

In [ ]:
y = unique_df['target'].copy()
X = unique_df.drop(columns=['target','existence_confidence'], axis=1).copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=1, stratify=y)
X_t, X_val, y_t, y_val = train_test_split(X, y, test_size=.25, random_state=2, stratify=y)

In [56]:
X_train

,lemmed_tweets,stemmed_tweets
3601,lward snow cover washington dc hand obama promise end do year climategate tcot fb,lward snow cover washington dc hand obama promis end did year climateg tcot fb
411,italy phd programme science management phd programme science management c,itali phd programm scienc manag phd programm scienc manag c
5949,debate discourage consider human intelligence gw nutcase remain unpersuaded logic evidence,debat discourag consid human intellig gw nutcas remain unpersuad logic evid
1170,china key fix warmingwith rapid expansion come sizable environmental impact world,china key fix warmingwith rapid expans come sizabl environment impact world
591,iceland volcano unlikely slow scientist afp paris • à big volcanic eruption cool,iceland volcano unlik slow scientist afp pari • à big volcan erupt cool
...,...,...
3569,scientific consensus melt way faster glacier tcot,scientif consensu melt way faster glacier tcot
2577,look great medium material human impact check witness story wwf,look great media materi human impact check wit stori wwf
3547,naomi klein cspan cause island extinction year,naomi klein cspan caus island extinct year
1215,report identifies disease health issue affctd u inclde mental health evrythings stake,report identifi diseas health issu affctd u incld mental health evryth stake


In [57]:
y_train

3601    1
411     2
5949    0
1170    1
591     1
       ..
3569    2
2577    1
3547    1
1215    1
3486    0
Name: target, Length: 4110, dtype: int64

### Let's get to modeling. First let's test out using SMOTE to eliminate our class imbalance.

In [81]:
imb_params= {
'sm__sampling_strategy' : ['minority', 'not_majority'],
'count__decode_error':['ignore', 'replace'],
'count__analyzer' : ['word', 'char', 'char_wb'],
'model__solver' : ['sag', 'saga'],
'model__max_iter' :[4000]}

In [82]:
imb_pipe = imbPipeline([('count', CountVectorizer()),
                        ('sm', SMOTE()),
                        ('model', LogisticRegression())])

In [83]:
imb_gs = GridSearchCV(imb_pipe, param_grid=imb_params, cv = 5, verbose = 5, n_jobs = -1)

In [84]:
imbmodel = imb_gs.fit(X_train['lemmed_tweets'], y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.60243309        nan 0.60364964        nan 0.60462287        nan
 0.6053528         nan 0.43090024        nan 0.43284672        nan
 0.43017032        nan 0.43041363        nan 0.43722628        nan
 0.43722628        nan 0.43722628        nan 0.4377129         nan]
  warnings.warn(


In [85]:
imb_best = imbmodel.best_estimator_

imby_trn_pred = imb_best.predict(X_train['lemmed_tweets'])
imby_tst_pred = imb_best.predict(X_test['lemmed_tweets'])

    
print('\t\tThe Train Results')
print(classification_report(y_train, imby_trn_pred))
print('\n\t\tThe Test Results')
print(classification_report(y_test, imby_tst_pred))

		The Train Results
              precision    recall  f1-score   support

           0       0.84      0.94      0.89       766
           1       0.93      0.94      0.94      2100
           2       0.94      0.85      0.89      1244

    accuracy                           0.91      4110
   macro avg       0.90      0.91      0.91      4110
weighted avg       0.92      0.91      0.91      4110


		The Test Results
              precision    recall  f1-score   support

           0       0.43      0.62      0.51       256
           1       0.73      0.66      0.69       700
           2       0.55      0.49      0.52       415

    accuracy                           0.60      1371
   macro avg       0.57      0.59      0.57      1371
weighted avg       0.62      0.60      0.61      1371



### Ok, we have not bad results but definitely overfit and not our best. Since we used SMOTE to address the class imbalnce, ideally now we would focus on accuracy since false negatives and positives bear the same weight. Now let's try it without SMOTE.

In [38]:
cv_params= {
'count__decode_error':['strict', 'ignore', 'replace'],
'count__analyzer' : ['word', 'char', 'char_wb'],
'count__max_df' : [.95],
'count__min_df' : [.05],
'model__solver' : ['lbfgs','sag', 'saga'],
'model__max_iter' :[4000]}
    

first_pipe = Pipeline([('count', CountVectorizer()),
                    ('model', LogisticRegression())])

In [39]:
cv_gs = GridSearchCV(first_pipe, param_grid=cv_params, cv = 5, verbose = 5, n_jobs = -1)

In [40]:
lrmodel = cv_gs.fit(X_train['lemmed_tweets'], y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


In [41]:
lr_best = lrmodel.best_estimator_

fy_trn_pred = lr_best.predict(X_train['lemmed_tweets'])
fy_tst_pred = lr_best.predict(X_test['lemmed_tweets'])

    
print('\t\tThe Train Results')
print(classification_report(y_train, fy_trn_pred))
print('\n\t\tThe Test Results')
print(classification_report(y_test, fy_tst_pred))

		The Train Results
              precision    recall  f1-score   support

           0       0.44      0.06      0.10       766
           1       0.53      0.95      0.68      2100
           2       0.50      0.09      0.15      1244

    accuracy                           0.52      4110
   macro avg       0.49      0.37      0.31      4110
weighted avg       0.50      0.52      0.41      4110


		The Test Results
              precision    recall  f1-score   support

           0       0.40      0.06      0.11       256
           1       0.53      0.94      0.67       700
           2       0.43      0.08      0.13       415

    accuracy                           0.52      1371
   macro avg       0.45      0.36      0.31      1371
weighted avg       0.47      0.52      0.41      1371



### Holy cajolly wildly overfit to training buuuut a start. Now for my random forest classifier search it took a bit of time as well so be fair warned.

In [42]:
#Param setup for Gridsearch
tf_params = {
 'tf__max_features':[100, 500, 2000, None],
 'tf__ngram_range': [(1, 1), (1, 2), (2, 2), None],
 'mnb__alpha': [.1, .5, .8, 1],
 'mnb__fit_prior': [True, False]}

In [43]:
mnb_pipe = Pipeline([('tf', TfidfVectorizer()),
                    ('mnb', MultinomialNB())])
mnb_gs = GridSearchCV(mnb_pipe, param_grid=tf_params, cv = 5, verbose = 1, n_jobs = -1)

In [44]:
mnb_model = mnb_gs.fit(X_train['lemmed_tweets'], y_train)

Fitting 5 folds for each of 128 candidates, totalling 640 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.58783455 0.58588808 0.55596107        nan 0.62068127 0.61849148
 0.56618005        nan 0.62992701 0.63333333 0.58199513        nan
 0.63284672 0.63430657 0.61216545        nan 0.46423358 0.46472019
 0.27664234        nan 0.57518248 0.57518248 0.35523114        nan
 0.60559611 0.60389294 0.43260341        nan 0.63309002 0.63576642
 0.54476886        nan 0.586618   0.58588808 0.55620438        nan
 0.62189781 0.62092457 0.56836983        nan 0.63430657 0.63600973
 0.58783455        nan 0.62822384 0.62822384 0.61119221        nan
 0.46447689 0.46374696 0.27639903        nan 0.57542579 0.5756691
 0.35717762        nan 0.61411192 0.6053528  0.43698297        nan
 0.63211679 0.64793187 0.54841849        nan 0.58686131 0.58832117
 0.55717762        nan 0.62141119 0.61678832 0.56788321        nan
 0.63187348 0.63065693 0.5892944         nan 0.6197080

In [45]:
mnb_best = mnb_model.best_estimator_

mnby_trn_pred = mnb_best.predict(X_train['lemmed_tweets'])
mnby_tst_pred = mnb_best.predict(X_test['lemmed_tweets'])


    
print('\t\tThe Train Results')
print(classification_report(y_train, mnby_trn_pred))
print('\n\t\tThe Test Results')
print(classification_report(y_test, mnby_tst_pred))

		The Train Results
              precision    recall  f1-score   support

           0       0.89      0.93      0.91       766
           1       0.94      0.93      0.94      2100
           2       0.90      0.89      0.89      1244

    accuracy                           0.92      4110
   macro avg       0.91      0.92      0.91      4110
weighted avg       0.92      0.92      0.92      4110


		The Test Results
              precision    recall  f1-score   support

           0       0.60      0.51      0.55       256
           1       0.72      0.78      0.75       700
           2       0.61      0.58      0.59       415

    accuracy                           0.67      1371
   macro avg       0.64      0.62      0.63      1371
weighted avg       0.66      0.67      0.66      1371



In [46]:
rf_params = {
 'tf__ngram_range': [(1, 2)],
 'tf__max_features':[100, 500, 2000, None],
 'tf__ngram_range': [(1, 1), (1, 2), (2, 2), None],
 'tf__min_df':[.05, None],
 'tf__max_df':[.95, None],
 'tf__stop_words': [None, 'english'],
 'rf__max_depth': [100, 500, 1000],
 'rf__min_samples_split': [100],
 'rf__max_leaf_nodes': [None]}

rf_pipe = Pipeline([('tf',  TfidfVectorizer()),
                     ('rf', RandomForestClassifier())])

In [47]:
rf_gs = GridSearchCV(rf_pipe, param_grid=rf_params, cv = 5, verbose = 1, n_jobs = -1)
rf_model = rf_gs.fit(X_train['lemmed_tweets'], y_train)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.51094891 0.51094891 0.51094891 0.51094891        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan 0.51094891 0.51094891
 0.51094891 0.51094891        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan 0.51094891 0.51094891 0.51094891 0.51094891
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.51094891 0.51094891 0.51094891 0.51094891        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        n

In [48]:
rfy_trn_pred = rf_model.predict(X_train['lemmed_tweets'])
rfy_tst_pred = rf_model.predict(X_test['lemmed_tweets'])
    
print('\t\tThe Train Results')
print(classification_report(y_train, rfy_trn_pred))
print('\n\t\tThe Test Results')
print(classification_report(y_test, rfy_tst_pred))

		The Train Results
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       766
           1       0.51      1.00      0.68      2100
           2       0.00      0.00      0.00      1244

    accuracy                           0.51      4110
   macro avg       0.17      0.33      0.23      4110
weighted avg       0.26      0.51      0.35      4110


		The Test Results
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       256
           1       0.51      1.00      0.68       700
           2       0.00      0.00      0.00       415

    accuracy                           0.51      1371
   macro avg       0.17      0.33      0.23      1371
weighted avg       0.26      0.51      0.35      1371



/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metr

In [49]:
confusion_matrix(y_test, rfy_tst_pred)

array([[  0, 256,   0],
       [  0, 700,   0],
       [  0, 415,   0]])

## Now that we have picked the best model lets get to vectorizing and our predictions.

In [50]:
# Fit the vectorizer on X_train["text"] and transform it
tfidf = TfidfVectorizer(max_features=100)
X_train_vectorized = tfidf.fit_transform(X_train['lemmed_tweets'])
Xtv_df = pd.DataFrame(X_train_vectorized.toarray(), columns=tfidf.get_feature_names())
Xtv_df

,action,agency,air,al,allergy,april,bad,believe,blame,blizzard,...,use,video,volcano,want,washington,way,weather,winter,world,year
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.383548,0.000000,0.0,0.0,0.000000,0.383548
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.436766,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.738926,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.607029,0.0,0.0,0.000000,0.000000
4106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.750493
4108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000


In [51]:
idf_values = dict(zip(tfidf.get_feature_names(), tfidf.idf_))
idf_values

{'action': 5.560221471204316,
 'agency': 5.278370319063328,
 'air': 5.514759097127559,
 'al': 4.964712760208286,
 'allergy': 5.537231952979617,
 'april': 5.429601288787252,
 'bad': 4.706301070056619,
 'believe': 5.514759097127559,
 'blame': 5.260978576351459,
 'blizzard': 4.855513468243295,
 'blog': 5.492780190408784,
 'bolivia': 5.514759097127559,
 'california': 5.351129673345756,
 'call': 5.227077024675777,
 'carbon': 5.260978576351459,
 'cause': 4.602922715602784,
 'claim': 5.537231952979617,
 'climatechange': 4.9393949522239975,
 'climategate': 5.370177868316451,
 'clinical': 5.429601288787252,
 'cochabamba': 5.560221471204316,
 'cold': 5.351129673345756,
 'come': 5.278370319063328,
 'conference': 4.964712760208286,
 'day': 4.778126804627874,
 'dc': 4.37266169651971,
 'debate': 5.260978576351459,
 'do': 5.030962145749487,
 'dont': 5.429601288787252,
 'earth': 4.736454108227306,
 'effect': 5.351129673345756,
 'energy': 4.4851396799464,
 'environment': 5.514759097127559,
 'environmen

In [52]:
X_t_vec = tfidf.fit_transform(X_train['lemmed_tweets'])
X_t_vec = pd.DataFrame.sparse.from_spmatrix(X_t_vec)
X_t_vec.columns = sorted(tfidf.vocabulary_)
X_t_vec.set_index(y_t.index, inplace=True)
X_t_vec

,action,agency,air,al,allergy,april,bad,believe,blame,blizzard,...,use,video,volcano,want,washington,way,weather,winter,world,year
1384,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.383548,0.000000,0.0,0.0,0.000000,0.383548
1402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
5669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.436766,0.000000
4298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.738926,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.607029,0.0,0.0,0.000000,0.000000
4899,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.750493
1329,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000


In [53]:
X_val_vec = tfidf.transform(X_val)
X_val_vec = pd.DataFrame.sparse.from_spmatrix(X_val_vec)
X_val_vec.columns = sorted(tfidf.vocabulary_)
#X_val_vec.set_index(y_val.index, inplace=True)
X_val_vec

,action,agency,air,al,allergy,april,bad,believe,blame,blizzard,...,use,video,volcano,want,washington,way,weather,winter,world,year
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
mnb = MultinomialNB()
mnb.fit(X_t_vec, y_t)
y_pred=mnb.predict(X_val_vec)
print(accuracy_score(y_test,y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [1371, 2]

In [ ]:
unique_df

In [ ]:
#y_hat = mnb.predict(X_val_vec)
#print(f1_score(y_val, y_hat))

In [ ]:
#model.fit(X_train,y_train)
#y_pred=model.predict(X_test)
#print(accuracy_score(y_test,y_pred))
#print(X_train.toarray()[:2])

In [ ]:
#deprecated funcs and values

def classify(self, input):
    input_text = input_vectorizer(max_features=100)
    input_counts = input_vectorizer.transform(input_text)
    predictions = self.classifier.predict(input_counts)
    print(predictions)
    
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
